In [ ]:
#Libraries that we used
import nltk
import numpy as np
import re

#Download necessary corpora
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#From these libraries import necessary files
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate import Alignment, AlignedSent
from nltk.translate.ibm1 import IBMModel1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Create arrays for all the languages
english_sentences = []
turkish_sentences = []

azerbaijani_sentences = []
turkish2_sentences= []

#Create a regex that changes words such as won't -> will not...
replacement_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', r'\g<1> will'),
    (r'(\w+)n\'t', r'\g<1> not'),
    (r'(\w+)\'ve', r'\g<1> have'),
    (r'(\w+)\'s', r'\g<1> is'),
    (r'(\w+)\'re', r'\g<1> are'),
    (r'(\w+)\'d', r'\g<1> would')
]

#This function taks a text and regex and changes words if any of them caught by the regex
def apply_regex_patterns(text, replacement_patterns):
  for pattern, replacement in replacement_patterns:
    text = re.sub(pattern, replacement, text)
  return text

#Read english_turkish_corpus_final.txt and fill english_sentences, turkish_sentences arrays
with open("english_turkish_corpus_final.txt", "r", encoding="utf-8") as file:
  for line in file:
    line = line.strip()
    if line:
      english, turkish = line.split("*")

      #Modify English sentence
      #Remove all the unnecessary signs
      english = re.sub(r'[()\[\]{}.,;!?\\-]', '', english)
      english = english.replace('[', '')
      english = english.replace(']', '')
      english = english.replace('.', '')
      english = english.strip()
      english = english.lower()
      english = apply_regex_patterns(english, replacement_patterns)

      #Modify Turkish sentence
      turkish = re.sub(r'[()\[\]{}.,;!?\\-]', '', turkish)
      turkish = turkish.replace('[', '')
      turkish = turkish.replace(']', '')
      turkish = turkish.replace('.', '')
      turkish = turkish.strip()
      turkish = turkish.lower()
      turkish = apply_regex_patterns(turkish, replacement_patterns)

      #Append the new sentences into corresponding arrays
      english_sentences.append(english.strip())
      turkish_sentences.append(turkish.strip())


with open("turkish_azerbaijani_corpus_final.txt", "r", encoding="utf-8") as file:
  for line in file:
    line = line.strip()
    if line:
      turkish2, azerbaijani = line.split("*")

      #Modify Turkish sentence
      turkish2 = re.sub(r'[()\[\]{}.,;!?\\-]', '', turkish2)
      turkish2 = turkish2.replace('[', '')
      turkish2 = turkish2.replace(']', '')
      turkish2 = turkish2.replace('.', '')
      turkish2 = turkish2.strip()
      turkish2 = turkish2.lower()
      turkish2 = apply_regex_patterns(turkish2, replacement_patterns)

      #Modify Azeri sentence
      azerbaijani = re.sub(r'[()\[\]{}.,;!?\\-]', '', azerbaijani)
      azerbaijani = azerbaijani.replace('[', '')
      azerbaijani = azerbaijani.replace(']', '')
      azerbaijani = azerbaijani.replace('.', '')
      azerbaijani = azerbaijani.strip()
      azerbaijani = azerbaijani.lower()
      azerbaijani = apply_regex_patterns(azerbaijani, replacement_patterns)

      #Append the new sentences into corresponding arrays
      azerbaijani_sentences.append(azerbaijani.strip())
      turkish2_sentences.append(turkish2.strip())

In [ ]:
def find_optimal_match(input_sentence, src_sentences, target_sentences):
  #Use TF-IDF vectorizer to convert sentences to vectors
  vectorizer = TfidfVectorizer(stop_words='english')
  sentence_vectors = vectorizer.fit_transform(src_sentences + [input_sentence])

  #Find the similarity between the input sentence and the corpus
  input_vector = sentence_vectors[-1]
  score_of_similarity = np.dot(sentence_vectors[:-1], input_vector.T).toarray().flatten()

  #Sort the similarity scores and get the top 10 sentences
  top_place = np.argsort(score_of_similarity)[-10:]

  #Retrieve the top 10 similar sentences and their Target translations
  similar_sentences = [src_sentences[i] for i in top_place]
  target_sentences = [target_sentences[i] for i in top_place]

  #Turn them into tokenize versions
  target_tokens = [nltk.word_tokenize(s) for s in target_sentences]
  similar_tokens = [nltk.word_tokenize(s) for s in similar_sentences]
  #Add them to list
  optimal_sentences = list(zip(similar_tokens, target_tokens))
  
  print("similar sentences in the corpus:\n", optimal_sentences)
  return optimal_sentences  #Return the top 10 similar source sentences

In [ ]:
#This function takes an array of sentences and adds them into Alignment Table so that every word has an alignment
def convert_to_aligned_sents(parallel_sentences):
  aligned_sents = []
  for initial_tokens, target_tokens in parallel_sentences:
    #Convert the initial tokens and target tokens into sentences by joining the tokens with spaces:
    initial_sentence = ' '.join(initial_tokens)
    target_sentence = ' '.join(target_tokens)

    #Create an alignment object called alignment by mapping each token position to itself using a list comprehension
    alignment = Alignment([(i, i) for i in range(min(len(initial_tokens), len(target_tokens)))])
    aligned_sent = AlignedSent(initial_tokens, target_tokens, alignment)
    #Append the aligned sentence to the aligned_sents list
    aligned_sents.append(aligned_sent)
  return aligned_sents

In [ ]:
def translate_sentence(sentence):
  # Tokenize the input sentence
  sentence_tokens = nltk.word_tokenize(sentence)

  # Create a set to keep track of checked words
  translation_tokens = []
  checked_words = set()

  #Iterate over each token in the sentence tokens
  for token in sentence_tokens:
    #Check if the token has already been checked
    if token in checked_words:
      #If it has been checked, append an empty string to the translation tokens list and continue to the next token
      translation_tokens.append("")
      continue
    
    #Initialize variables for the best translation and maximum probability
    best_translation = ""
    max_prob = 0.0

    # Iterate over each translation token for the current token
    for idx, tr_token in enumerate(ibm1.translation_table[token]):

      #Check if the probability of the translation token is greater than the current maximum probability
      if ibm1.translation_table[token][tr_token] > max_prob:
        #If it is greater, update the maximum probability and set the best translation to the current translation token
        max_prob = ibm1.translation_table[token][tr_token]
        best_translation = tr_token

      #If the probability of the translation token is equal to the current maximum probability
      elif ibm1.translation_table[token][tr_token] == max_prob:
        #Check if the best translation is empty
        if not best_translation:
          #If it is empty, set the best translation to the current translation token
          best_translation = tr_token
        else:
          #If it is not empty, append the current translation token to the best translation with a space separator
          best_translation = best_translation + " " + tr_token
    
    # Check if a best translation was found
    if best_translation:
        #If a best translation was found, append it to the translation tokens list and add it to the checked words set
        translation_tokens.append(best_translation)
        checked_words.add(best_translation)

  #Return the final translation tokens
  return translation_tokens

In [ ]:
#Remove substrings
def remove_repeating_variables(array):
  unique_variables = []
  #If the given array has not unique variables, remove them
  for item in array:
    if item not in unique_variables:
      unique_variables.append(item)
  return ' '.join(unique_variables)

In [ ]:
#Ask the user a question and save the answer
user_option = input("Please select 1 or 2: \n 1 For English to Turksih \n 2 For Azerbaijani to Turkish \n")

#If the answer is not 1 or 2 ask him again
while user_option != '1' and user_option != '2':
  user_option = input("Please select 1 or 2: \n 1 For English to Turksih \n 2 For Azerbaijani to Turkish \n")

#If the answer is 1 call the necessasary functions for English to Turkish translations
if( int(user_option) == 1):
  user_sentence = input("Please enter an English sentence for translation to Turkish \n")
  print("Your sentence: ", user_sentence)
  parallel_sentences = find_optimal_match(user_sentence, english_sentences, turkish_sentences)
  aligned_sents = convert_to_aligned_sents(parallel_sentences)
  ibm1 = IBMModel1(aligned_sents, iterations=10)

  translation = translate_sentence(user_sentence)
  fixed_translation = remove_repeating_variables(translation)
  print("Input:", user_sentence)
  print("Translation:", fixed_translation)

#Else if the answer is 2 call the necessasary functions for Azerbaijani to Turkish translations
elif( int(user_option) == 2):
  user_sentence = input("Please enter an Azerbaijani sentence for translation to Turkish \n")
  parallel_sentences = find_optimal_match(user_sentence, azerbaijani_sentences, turkish2_sentences)
  aligned_sents = convert_to_aligned_sents(parallel_sentences)
  ibm1 = IBMModel1(aligned_sents, iterations=10)

  translation = translate_sentence(user_sentence)
  fixed_translation = remove_repeating_variables(translation)
  print("Input:", user_sentence)
  print("Translation:", fixed_translation)

Please select 1 or 2: 
 1 For English to Turksih 
 2 For Azerbaijani to Turkish 
2
Please enter an Azerbaijani sentence for translation to Turkish 
Bu kitablar mənim sözümü tutmamaqda ittiham etdilər
similar sentences in the corpus:
 [(['bunu', 'dörd', 'gündə', 'etdilər'], ['bunu', 'dört', 'günde', 'yaptılar']), (['1992ci', 'ildə', 'kitablar', 'nəşr', 'etməyə', 'başladılar'], ['1992', 'woulde', 'kitap', 'yayınlamaya', 'başladılar']), (['kitabxanamızda', 'çoxlu', 'ingiliscə', 'kitablar', 'var'], ['kütüphanemizde', 'çok', 'sayıda', 'i̇ngilizce', 'kitap', 'var']), (['sözümü', 'kəsmə', 'danışdığımı', 'görmürsən'], ['sözümü', 'kesme', 'konuştuğumu', 'görmüyor', 'musun']), (['adam', 'məni', 'məsuliyyətsizlikdə', 'ittiham', 'etdi'], ['adam', 'beni', 'sorumsuz', 'olmakla', 'suçladı']), (['nə', 'olursa', 'olsun', 'sözümü', 'tutacam'], ['ne', 'olursa', 'olsun', 'sözümü', 'tutacağım']), (['o', 'onu', 'ehtiyatsızlıqda', 'ittiham', 'edib'], ['onu', 'dikkatsiz', 'olmakla', 'suçladı']), (['belə', 'ki